<a href="https://colab.research.google.com/github/tomonari-masada/course2025-sml/blob/main/06_linear_regression_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 重回帰による住宅価格の予測

* California housing datasetという有名なデータセットを使う。

  * scikit-learnからロードできるバージョンは、前処理が済んだキレイなデータ。
  * ここでは、前処理も自分でおこなってみる。

* データの前処理については、下記の本の第2章を参考にした。
  * [Aurélien Géron. Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition.](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

* 機械学習において線形回帰モデルを使うときは、予測性能が重要。
  * 予測性能が上がるなら何でもする、という考え方。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error

%config InlineBackend.figure_format = 'retina'

## データセットの取得

In [ ]:
!wget https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz
!tar zxvf housing.tgz

--2024-05-18 01:30:25--  https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409488 (400K) [application/octet-stream]
Saving to: ‘housing.tgz’

housing.tgz         100%[===================>] 399.89K  --.-KB/s    in 0.04s   

2024-05-18 01:30:25 (11.1 MB/s) - ‘housing.tgz’ saved [409488/409488]

housing.csv


In [ ]:
df = pd.read_csv("housing.csv")

In [ ]:
df.head()

In [ ]:
df.info()

* 数値ではない特徴量が一つだけある
  * こういう特徴量を、カテゴリカルな特徴量と呼ぶ。

In [ ]:
df['ocean_proximity'].value_counts()

* 今回は、one-hot encodingで数値化する。
  * pandasのget_dummiesメソッドを利用する。

In [ ]:
df_onehot = pd.get_dummies(df, dtype=int)

In [ ]:
df_onehot.head()

In [ ]:
df_onehot.info()

## 問題設定
* median_house_valueを予測する。
 * これ以外は特徴量として使う。

In [ ]:
X = df_onehot.drop('median_house_value', axis=1)
y = df_onehot["median_house_value"].copy()

In [ ]:
X.head()

## 実験のための準備

### 訓練データ/検証データ/テストデータ

* 訓練データ、検証データ、テストデータに分ける
  * 今回は、6:2:2になるように分ける。（この比率に深い意味はない。）
  * 今回は、交差検証は行わない。

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=1234)

In [ ]:
print(X_train.shape, X_valid.shape, X_test.shape)

In [ ]:
X_train.info()

In [ ]:
X_valid.info()

In [ ]:
X_test.info()

### 欠測値への対応

* total_bedroomsの値が欠けているエントリがあるので・・・
* 今回は、中央値で埋めることにする。（ここを変更してもいいです。）
* 中央値のような統計量を得るときは、訓練データのみを使う。
  * 検証データやテストデータで、平均値や中央値などの統計量を調べてはいけない。


In [ ]:
X_train["total_bedrooms"].hist(bins=50);

In [ ]:
total_bedrooms_median = X_train["total_bedrooms"].median()
print(total_bedrooms_median)
total_bedrooms_mean = X_train["total_bedrooms"].mean()
print(total_bedrooms_mean)

* 問題: 中央値と平均値がかなり違う。この違いは、何を意味するか。

In [ ]:
X_train = X_train.fillna({'total_bedrooms': total_bedrooms_median})

In [ ]:
X_train.info()

In [ ]:
X_valid = X_valid.fillna({'total_bedrooms': total_bedrooms_median})
X_valid.info()

In [ ]:
X_test = X_test.fillna({'total_bedrooms': total_bedrooms_median})
X_test.info()

## 訓練データのEDA
* EDA = exploratory data analysis

In [ ]:
X_train.describe()

### ヒストグラム

In [ ]:
X_train.hist(bins=50, figsize=(15,12));

* total_roomsの値の分布に注目してみる

In [ ]:
plt.hist(X_train.total_rooms, bins=50);

* 説明変数の値がどのように分布するかは、回帰モデルの予測性能に直接は関係しない。
* 回帰モデルでは、誤差項が正規分布に従うという仮定はすることがある。
* しかし、説明変数の値の分布については、何も仮定しない。
  * 例えば、0か1かの2通りの値しかとらない説明変数もよく使う。
* とはいえ、それで予測性能が上がるなら、説明変数の値の分布を変更してみる余地はある。

* total_roomsの分布を、正規分布っぽい形に近づけてみる。

In [ ]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()
pt.fit(X_train[["total_rooms"]])
X_train["pt_total_rooms"] = pt.transform(X_train[["total_rooms"]])

In [ ]:
X_train["pt_total_rooms"].hist(bins=50);

* このように分布を変えた方が予測性能は良くなるだろうか？

* とりあえず、追加した列を削除し、元に戻しておく。

In [ ]:
X_train = X_train.drop(columns="pt_total_rooms")

* housing_median_ageを見てみる
 * 最大値の頻度が妙に高い。

In [ ]:
X_train.housing_median_age.hist(bins=50);

In [ ]:
X_train.housing_median_age.value_counts().sort_index(ascending=False).head()

### 相関係数
* 予測目的で線形回帰モデルを使う場合、いわゆる多重線形性は、さほど気にしなくてよい。
  * 検定に線形回帰モデルを使う場合は、気にしないといけない。
  * 多重共線性 https://bellcurve.jp/statistics/glossary/1792.html

* 相関係数は、最小値-1、最大値1。

In [ ]:
plt.subplots(figsize=(10,10))
sns.heatmap(X_train.corr(), vmin=-1, vmax=1, annot=True, square=True);

### ペア・プロット

* one-hot encodingは除外してプロットする。

In [ ]:
X_train.drop(columns=X_train.filter(regex=("ocean_*")))

In [ ]:
sns.pairplot(X_train.drop(columns=X_train.filter(regex=("ocean_*"))), diag_kind='kde');

## 線形回帰

* training setでモデルのパラメータを推定する


In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)

* validation set上で評価する


In [ ]:
y_valid_pred = reg.predict(X_valid)
rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
print(f'RMSE: {rmse:f}')

* ここで、訓練データのターゲット（目的変数）の値の分布を見てみる

In [ ]:
y_train.hist(bins=50);

In [ ]:
y_train.value_counts().head()

* 予測値が、訓練データ内でのターゲットの最大値を超えないようにして、再び評価する

In [ ]:
y_valid_pred = reg.predict(X_valid)
y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
print(f'RMSE: {rmse:f}')

## リッジ回帰
* 係数の二乗の和を抑える正則化を含む。

In [ ]:
reg = Ridge(alpha=1.0)
reg.fit(X_train, y_train)
y_valid_pred = reg.predict(X_valid)
y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
print('RMSE: {:f}'.format(mean_squared_error(y_valid, y_valid_pred, squared=False)))

## Lasso
* 係数の絶対値の和を抑える正則化を含む。

In [ ]:
reg = Lasso(alpha=1.0, max_iter=5000)
reg.fit(X_train, y_train)
y_valid_pred = reg.predict(X_valid)
y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
print('RMSE: {:f}'.format(mean_squared_error(y_valid, y_valid_pred, squared=False)))

## 試行錯誤するための選択肢

### 説明変数の値を加工

* スケーラのfitは訓練データのみで行う

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# スケーラのfitは訓練データのみで行う
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = X_train.copy()
X_train_scaled[X_train.columns] = scaler.transform(X_train)

X_valid_scaled = X_valid.copy()
X_valid_scaled[X_valid.columns] = scaler.transform(X_valid) # 検証データに同じスケーリングを適用

X_test_scaled = X_test.copy()
X_test_scaled[X_test.columns] = scaler.transform(X_test) # 検証データに同じスケーリングを適用

In [ ]:
X_train_scaled.describe()

### 正則化パラメータをチューニング

In [ ]:
for alpha in 10. ** np.arange(-6, 7):
  reg = Ridge(alpha=alpha, random_state=123)
  reg.fit(X_train_scaled, y_train)
  y_valid_pred = reg.predict(X_valid_scaled)
  y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
  rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
  print(f'alpha: {alpha:f}; RMSE: {rmse:f}')

In [ ]:
reg = LinearRegression()
reg.fit(X_train_scaled, y_train)
y_valid_pred = reg.predict(X_valid_scaled)
y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
print(f'RMSE: {rmse:f}')

### 試行錯誤の例：新しい説明変数を作成

* 下の例では、何をしているだろうか？

In [ ]:
print(X_train.longitude.median(), X_train.latitude.median())

In [ ]:
med_lon = X_train.longitude.median()
med_lat = X_train.latitude.median()

In [ ]:
flag_lon = (X_train.longitude < med_lon) * 1
flag_lat = (X_train.latitude < med_lat) * 1

X_train['lon'] = flag_lon
X_train['lat'] = flag_lat

In [ ]:
X_train.describe()

In [ ]:
flag_lon = (X_valid.longitude < med_lon) * 1
flag_lat = (X_valid.latitude < med_lat) * 1

X_valid['lon'] = flag_lon
X_valid['lat'] = flag_lat

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train) # スケーラのfitは訓練データのみで行う
X_train_scaled = X_train.copy()
X_train_scaled[X_train.columns] = scaler.transform(X_train)
X_valid_scaled = X_valid.copy()
X_valid_scaled[X_valid.columns] = scaler.transform(X_valid) # 検証データに同じスケーリングを適用

In [ ]:
reg = LinearRegression()
reg.fit(X_train_scaled, y_train)
y_valid_pred = reg.predict(X_valid_scaled)
y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
print(f'RMSE: {rmse:f}')

# 課題

* RMSEによって評価される予測性能を、良くして下さい
* test setとそれ以外の部分の分割は、変えないでください
  * test set以外の部分をどう使うかは、自由です。
  * training setとvalidation setをくっつけて、交差検証をしていいです。
* その他、いろいろ試行錯誤してみてください。
  * リッジ回帰とLassoを使ってもいいです
  * 高次多項式特徴量を使ってもいいです（cf. `sklearn.preprocessing.PolynomialFeatures`）
* 予測手法のチューニングを尽くした上で、最後にtest setでのRMSEによる評価を実施してください。
  * test setでの評価結果を見て、チューニングに戻ってはいけません。